### Moduļi

In [191]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator
from scipy.linalg import expm, logm

### Konstantes

In [192]:
THETA = np.pi / 4

EPSILON = 1e-3

GATESET = {}
tempqc = QuantumCircuit(1)
tempqc.rz(THETA, 0)
GATESET['Rz'] = tempqc
tempqc = QuantumCircuit(1)
tempqc.h(0)
GATESET['H'] = tempqc
del tempqc

print(Operator(GATESET['H']).data)
print(Operator(GATESET['Rz']).data)

N_QUBITS = 1

# Pauli matrices
I_MATRIX = np.eye(2, dtype=complex)
X_MATRIX = np.array([[0, 1], [1, 0]], complex)
Y_MATRIX = np.array([[0, -1j], [1j, 0]], complex)
Z_MATRIX = np.array([[1, 0], [0, -1]], complex)
PAULI_BASE = [X_MATRIX, Y_MATRIX, Z_MATRIX]

[[ 0.70710678+0.j  0.70710678+0.j]
 [ 0.70710678+0.j -0.70710678+0.j]]
[[0.92387953-0.38268343j 0.        +0.j        ]
 [0.        +0.j         0.92387953+0.38268343j]]


### Saglabātās īsās ķēdes

In [193]:
SHORT_CIRCUITS = []

# all combinations of H and Rz with up to 4 gates
for i in range(1, 7):
    for combo in np.array(np.meshgrid(*[['H', 'Rz']] * i)).T.reshape(-1, i):
        name = ' '.join(combo)
        qc = QuantumCircuit(1)
        for gate in combo:
            qc.compose(GATESET[gate], [0], inplace=True)
        SHORT_CIRCUITS.append((name, qc, Operator(qc).data))

# test print the operators
for circuit in SHORT_CIRCUITS:
    print(circuit[0])

H
Rz
H H
H Rz
Rz H
Rz Rz
H H H
H Rz H
Rz H H
Rz Rz H
H H Rz
H Rz Rz
Rz H Rz
Rz Rz Rz
H H H H
H Rz H H
Rz H H H
Rz Rz H H
H H Rz H
H Rz Rz H
Rz H Rz H
Rz Rz Rz H
H H H Rz
H Rz H Rz
Rz H H Rz
Rz Rz H Rz
H H Rz Rz
H Rz Rz Rz
Rz H Rz Rz
Rz Rz Rz Rz
H H H H H
H Rz H H H
Rz H H H H
Rz Rz H H H
H H Rz H H
H Rz Rz H H
Rz H Rz H H
Rz Rz Rz H H
H H H Rz H
H Rz H Rz H
Rz H H Rz H
Rz Rz H Rz H
H H Rz Rz H
H Rz Rz Rz H
Rz H Rz Rz H
Rz Rz Rz Rz H
H H H H Rz
H Rz H H Rz
Rz H H H Rz
Rz Rz H H Rz
H H Rz H Rz
H Rz Rz H Rz
Rz H Rz H Rz
Rz Rz Rz H Rz
H H H Rz Rz
H Rz H Rz Rz
Rz H H Rz Rz
Rz Rz H Rz Rz
H H Rz Rz Rz
H Rz Rz Rz Rz
Rz H Rz Rz Rz
Rz Rz Rz Rz Rz
H H H H H H
H Rz H H H H
Rz H H H H H
Rz Rz H H H H
H H Rz H H H
H Rz Rz H H H
Rz H Rz H H H
Rz Rz Rz H H H
H H H Rz H H
H Rz H Rz H H
Rz H H Rz H H
Rz Rz H Rz H H
H H Rz Rz H H
H Rz Rz Rz H H
Rz H Rz Rz H H
Rz Rz Rz Rz H H
H H H H Rz H
H Rz H H Rz H
Rz H H H Rz H
Rz Rz H H Rz H
H H Rz H Rz H
H Rz Rz H Rz H
Rz H Rz H Rz H
Rz Rz Rz H Rz H
H H H Rz Rz H
H

### Ievads

In [194]:
U_target = np.sqrt(0.5) * np.array([
    [1, 1],
    [1j, -1j]
])
# vai unitāra?
print("Is unitary:", np.allclose(U_target @ U_target.conj().T, np.eye(2)))
print("U_target:")
print(U_target)

Is unitary: True
U_target:
[[0.70710678+0.j         0.70710678+0.j        ]
 [0.        +0.70710678j 0.        -0.70710678j]]


### Kods

In [195]:
def extract_axis_angle(U):
    tr = np.trace(U)
    # clamp real value to [-1,1] for numerical stability
    cos_half = np.real(tr) / 2.0
    cos_half = np.clip(cos_half, -1.0, 1.0)
    theta = 2.0 * np.arccos(cos_half)
    # If theta ~ 0, axis is ill-defined
    if np.isclose(theta, 0.0, atol=1e-12):
        return np.array([0.0, 0.0, 1.0]), 0.0
    s = np.sin(theta/2.0)
    # extract axis components: Tr(U sigma_i) = -2i s n_i  => n_i = Im(Tr(U sigma_i)) / (2 s)
    n = np.zeros(3)
    for i, P in enumerate(PAULI_BASE):
        n[i] = np.imag(np.trace(U @ P)) / (2.0 * s)
    n /= np.linalg.norm(n)
    return n, theta

def gc_decomposition(U):
    # Iegūst grupas komutatoru, kas aptuveni vienāds ar novirzi:

    U = logm(U)

    # determinantam jābūt 1
    if not np.isclose(np.linalg.det(U), 1.0, atol=1e-12):
        print("Adjusting determinant from", np.linalg.det(U), "to 1.")
        U = U / np.linalg.det(U)

    n_c, epsilon = extract_axis_angle(U)

    # Uzbūvē koriģējošo komutatoru
    temp = np.array([1.0, 0.0, 0.0])
    if abs(np.dot(temp, n_c)) > 0.9:
        temp = np.array([0.0, 1.0, 0.0])

    u = temp - np.dot(temp, n_c) * n_c
    u /= np.linalg.norm(u)
    v = np.cross(n_c, u)
    v /= np.linalg.norm(v)

    print("u:", u, "v:", v, "n:", n_c)

    alpha = beta = np.sqrt(epsilon)

    a = -1j * alpha/2 * (u[0]*PAULI_BASE[0] + u[1]*PAULI_BASE[1] + u[2]*PAULI_BASE[2])
    b = -1j * beta/2 * (v[0]*PAULI_BASE[0] + v[1]*PAULI_BASE[1] + v[2]*PAULI_BASE[2])

    A = expm(a)
    B = expm(b)

    print("Group commutator:")
    print(A @ B @ np.linalg.inv(A) @ np.linalg.inv(B))

    return A, B

def solovay_kitaev_decomposition(U_target, depth=3):
    # Meklē labāko tuvinājumu no īsajām ķēdēm

    if depth == 0:
        print("Depth 0 reached.")
        print("Target:")
        print(U_target)

        min_error = float('inf')
        best_circuit = None
        for name, qc, U in SHORT_CIRCUITS:
            error = np.linalg.norm(U_target - U, 2)
            if error < min_error:
                min_error = error
                best_circuit = (name, qc, U)

        U_approx = best_circuit[2]
        qc_approx = best_circuit[1]
        print("Approximate circuit:", best_circuit[0], "with error:", min_error)
        
        return qc_approx
    
    qc_U = solovay_kitaev_decomposition(U_target, depth-1)
    U_approx = Operator(qc_U).data

    residual = U_target @ U_approx.conj().T
    print("Residual operator:")
    print(residual)
    
    A, B = gc_decomposition(residual)

    qc_out = QuantumCircuit(1)
    # papildina ar A, B, A†, B†
    for gate in [A, B, A.conj().T, B.conj().T]:
        qc_gate = solovay_kitaev_decomposition(gate, depth-1)
        qc_out.compose(qc_gate, [0], inplace=True)
    # papildina ar sākotnējo tuvinošo ķēdi
    qc_out.compose(qc_U, [0], inplace=True)

    return qc_out

final_circuit = solovay_kitaev_decomposition(U_target, 1)

final_operator = Operator(final_circuit).data
print("Final circuit operator:")
print(final_operator)

print("Final approximation error:", np.linalg.norm(U_target - final_operator, 2))

final_circuit.draw()

Depth 0 reached.
Target:
[[0.70710678+0.j         0.70710678+0.j        ]
 [0.        +0.70710678j 0.        -0.70710678j]]
Approximate circuit: H Rz Rz with error: 0.7653668647301795
Residual operator:
[[ 7.07106781e-01+7.07106781e-01j  2.29934717e-17-2.29934717e-17j]
 [-2.29934717e-17+2.29934717e-17j  7.07106781e-01+7.07106781e-01j]]
Adjusting determinant from (-0.6168502750680849-8.719671245021581e-17j) to 1.
u: [-1.41357986e-16  1.00000000e+00  0.00000000e+00] v: [ 0. -0.  1.] n: [1.00000000e+00 1.41357986e-16 0.00000000e+00]
Group commutator:
[[ 0.27964771+0.58798536j -0.58798536-0.47994125j]
 [ 0.58798536-0.47994125j  0.27964771-0.58798536j]]
Depth 0 reached.
Target:
[[ 0.63233949-6.16297582e-33j -0.7746914 +1.09508816e-16j]
 [ 0.7746914 +1.09508816e-16j  0.63233949+0.00000000e+00j]]
Approximate circuit: H Rz Rz H Rz Rz with error: 0.7700448736863283
Depth 0 reached.
Target:
[[0.63233949-0.7746914j 0.        +0.j       ]
 [0.        +0.j        0.63233949+0.7746914j]]
Approximate

┌───┐┌─────────┐┌─────────┐┌───┐┌─────────┐┌─────────┐┌─────────┐┌───┐┌───┐»
q: ┤ H ├┤ Rz(π/4) ├┤ Rz(π/4) ├┤ H ├┤ Rz(π/4) ├┤ Rz(π/4) ├┤ Rz(π/4) ├┤ H ├┤ H ├»
   └───┘└─────────┘└─────────┘└───┘└─────────┘└─────────┘└─────────┘└───┘└───┘»
«   ┌─────────┐┌─────────┐┌─────────┐┌───┐┌─────────┐┌─────────┐┌───┐┌───┐┌───┐»
«q: ┤ Rz(π/4) ├┤ Rz(π/4) ├┤ Rz(π/4) ├┤ H ├┤ Rz(π/4) ├┤ Rz(π/4) ├┤ H ├┤ H ├┤ H ├»
«   └─────────┘└─────────┘└─────────┘└───┘└─────────┘└─────────┘└───┘└───┘└───┘»
«   ┌───┐┌─────────┐┌─────────┐
«q: ┤ H ├┤ Rz(π/4) ├┤ Rz(π/4) ├
«   └───┘└─────────┘└─────────┘